## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2021-04-25 14:07:59,-23.1203,-134.9692,77.14,76,60,6.82,light rain
1,1,Dikson,RU,2021-04-25 14:06:53,73.5069,80.5464,18.09,88,100,10.51,overcast clouds
2,2,Vaini,TO,2021-04-25 14:07:59,-21.2000,-175.2000,73.40,100,75,1.14,light rain
3,3,Faanui,PF,2021-04-25 14:08:00,-16.4833,-151.7500,80.40,67,6,4.32,clear sky
4,4,Chicama,PE,2021-04-25 14:08:00,-7.8447,-79.1469,62.60,94,75,6.91,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2021-04-25 14:07:59,-23.1203,-134.9692,77.14,76,60,6.82,light rain
2,2,Vaini,TO,2021-04-25 14:07:59,-21.2000,-175.2000,73.40,100,75,1.14,light rain
5,5,Arraial Do Cabo,BR,2021-04-25 14:07:07,-22.9661,-42.0278,78.80,78,20,2.30,few clouds
19,19,Avarua,CK,2021-04-25 14:08:06,-21.2078,-159.7750,75.20,88,94,12.66,light rain
22,22,Atuona,PF,2021-04-25 14:06:45,-9.8000,-139.0333,79.48,78,72,20.07,broken clouds
25,25,Mizdah,LY,2021-04-25 14:08:08,31.4451,12.9801,75.40,21,53,5.64,broken clouds
42,42,San Policarpo,PH,2021-04-25 14:08:16,12.1791,125.5072,78.94,79,81,10.47,broken clouds
59,59,Mareeba,AU,2021-04-25 14:08:22,-17.0000,145.4333,75.20,94,75,9.22,broken clouds
62,62,Bubaque,GW,2021-04-25 14:08:23,11.2833,-15.8333,78.55,51,28,6.62,scattered clouds
64,64,Port Alfred,ZA,2021-04-25 14:08:24,-33.5906,26.8910,76.66,56,97,16.11,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                112
City                   112
Country                110
Date                   112
Lat                    112
Lng                    112
Max Temp               112
Humidity               112
Cloudiness             112
Wind Speed             112
Current Description    112
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                110
City                   110
Country                110
Date                   110
Lat                    110
Lng                    110
Max Temp               110
Humidity               110
Cloudiness             110
Wind Speed             110
Current Description    110
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.14,light rain,-23.1203,-134.9692,
2,Vaini,TO,73.40,light rain,-21.2000,-175.2000,
5,Arraial Do Cabo,BR,78.80,few clouds,-22.9661,-42.0278,
19,Avarua,CK,75.20,light rain,-21.2078,-159.7750,
22,Atuona,PF,79.48,broken clouds,-9.8000,-139.0333,
25,Mizdah,LY,75.40,broken clouds,31.4451,12.9801,
42,San Policarpo,PH,78.94,broken clouds,12.1791,125.5072,
59,Mareeba,AU,75.20,broken clouds,-17.0000,145.4333,
62,Bubaque,GW,78.55,scattered clouds,11.2833,-15.8333,
64,Port Alfred,ZA,76.66,overcast clouds,-33.5906,26.8910,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.14,light rain,-23.1203,-134.9692,Pension Maro'i
2,Vaini,TO,73.40,light rain,-21.2000,-175.2000,Keleti Beach Resort
5,Arraial Do Cabo,BR,78.80,few clouds,-22.9661,-42.0278,Pousada Porto Praia
19,Avarua,CK,75.20,light rain,-21.2078,-159.7750,Paradise Inn
22,Atuona,PF,79.48,broken clouds,-9.8000,-139.0333,Villa Enata


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
WeatherPy_vacation = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(WeatherPy_vacation, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.3)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))